#### KDE

Here we used the KDE implementation of the library sk-learn.

-  [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html)


In [1]:
import pandas as pd
import numpy as np
import os 
import random
from Funciones import *
from scipy.spatial.distance import cdist
medida = 'euclidean'

random.seed(42)

import pygad 
from sklearn.neighbors import KernelDensity



### We load one of the datasets

In [2]:
# Paths and Embeddings for the training set
set = 'Mobio'
a,b = load_train(set)

Img_Con_train_paths = reshape_paths(a[0][0])
Aud_Con_train_paths = reshape_paths(a[0][1])
Mix_Con_train_paths = reshape_paths(a[0][2])

Img_Descon_train_paths = reshape_paths(a[1][0])
Aud_Descon_train_paths = reshape_paths(a[1][1])
Mix_Descon_train_paths = reshape_paths(a[1][2])


Img_Con_train_embeddings = reshape_embed(b[0][0])
Aud_Con_train_embeddings = reshape_embed(b[0][1])
Mix_Con_train_embeddings = reshape_embed(b[0][2])

Img_Descon_train_embeddings = reshape_embed(b[1][0])
Aud_Descon_train_embeddings = reshape_embed(b[1][1])
Mix_Descon_train_embeddings = reshape_embed(b[1][2])


# Paths and Embeddings for the validation set
a,b = load_validation(set)

Img_Con_validation_paths = reshape_paths(a[0][0])
Aud_Con_validation_paths = reshape_paths(a[0][1])
Mix_Con_validation_paths = reshape_paths(a[0][2])

Img_Descon_validation_paths = reshape_paths(a[1][0])
Aud_Descon_validation_paths = reshape_paths(a[1][1])
Mix_Descon_validation_paths = reshape_paths(a[1][2])


Img_Con_validation_embeddings = reshape_embed(b[0][0])
Aud_Con_validation_embeddings = reshape_embed(b[0][1])
Mix_Con_validation_embeddings = reshape_embed(b[0][2])

Img_Descon_validation_embeddings = reshape_embed(b[1][0])
Aud_Descon_validation_embeddings = reshape_embed(b[1][1])
Mix_Descon_validation_embeddings = reshape_embed(b[1][2])


# Paths and Embeddings for the test set
a,b = load_test(set)

Img_Con_test_paths = reshape_paths(a[0][0])
Aud_Con_test_paths = reshape_paths(a[0][1])
Mix_Con_test_paths = reshape_paths(a[0][2])

Img_Descon_test_paths = reshape_paths(a[1][0])
Aud_Descon_test_paths = reshape_paths(a[1][1])
Mix_Descon_test_paths = reshape_paths(a[1][2])

Img_Con_test_embeddings = reshape_embed(b[0][0])
Aud_Con_test_embeddings = reshape_embed(b[0][1])
Mix_Con_test_embeddings = reshape_embed(b[0][2])

Img_Descon_test_embeddings = reshape_embed(b[1][0])
Aud_Descon_test_embeddings = reshape_embed(b[1][1])
Mix_Descon_test_embeddings = reshape_embed(b[1][2])



##
Conocidos = len(Img_Con_train_embeddings)
Desconocidos = len(Img_Descon_train_embeddings)
muestras_img = len(Img_Con_train_embeddings[0])
muestras_aud = len(Aud_Con_train_embeddings[0])

img_test = len(Img_Con_test_embeddings[0])
aud_test = len(Aud_Con_test_embeddings[0])

columnas = a[0][0].columns

In [3]:
print ('Known: ', Conocidos)
print ('Unknown: ', Desconocidos)
print ('Images per person (train): ', muestras_img)
print ('Audios per person (train): ', muestras_aud)
print ('Images per person (test): ', img_test)
print ('Audios per person (test): ', aud_test)

Conocidos:  46
Desconocidos:  46
Muestras por imagen:  40
Muestras por audio:  2
Muestras por imagen test:  30
Muestras por audio test:  1


In [4]:
from sklearn.decomposition import PCA

Data_PCA = Mix_Con_train_embeddings
ShapedData = Data_PCA.reshape(Conocidos*muestras_img,1024)
ShapedData = np.array(ShapedData)

PCA_emb = PCA(n_components=.95)
PCA_emb.fit(ShapedData)
dim = PCA_emb.n_components_

### Start of the experiment

In [5]:
Data_train = Mix_Con_train_embeddings.reshape(Conocidos*muestras_img,1024)
Data_test_con = Mix_Con_test_embeddings.reshape(Conocidos*img_test,1024)
Data_val_con = Mix_Con_validation_embeddings.reshape(Conocidos*img_test,1024)

Data_test_descon = Mix_Descon_test_embeddings.reshape(Desconocidos*img_test,1024)
Data_val_descon = Mix_Descon_validation_embeddings.reshape(Desconocidos*img_test,1024)



In [6]:
kernel_dict =  {
    1 : 'gaussian',
    2 : 'exponential',
    3 : 'epanechnikov',
}

In [7]:
import math
def plot_outliers(kernel,banda,train,data_con,data_descon,title = 'Umbral GLOSH',percentile=100):
    Kern = KernelDensity(kernel=kernel, bandwidth=banda).fit(train)
    percentile = 100-percentile
    originals = Kern.score_samples(train)
    shaped_con = np.array(data_con).reshape(Conocidos*img_test,1024)
    shaped_descon = np.array(data_descon).reshape(Desconocidos*img_test,1024)
    con_scores = Kern.score_samples(shaped_con)
    descon_scores = Kern.score_samples(shaped_descon)
    
    # Convert the -inf values from the scores to the lowest finite number
    con_scores[con_scores == -np.inf] = np.min(con_scores[con_scores != -np.inf])
    descon_scores[descon_scores == -np.inf] = np.min(descon_scores[descon_scores != -np.inf])

    
    threshold = np.percentile(originals, percentile)
    print ("Threshold: ", threshold)
    outliers = np.where(originals > threshold)[0]
    inliers = np.where(originals <= threshold)[0]  
  

    
    plt.figure(figsize=(15,10))
    plt.hist (originals, bins = 100)
    plt.hist(con_scores, bins = 100)
    plt.hist(descon_scores, bins = 100)
    plt.plot([threshold, threshold], [0, 100], '--', color='k')
    plt.legend(['threshold','Train','Known','Unknown'])
    plt.title(title)
    plt.xlabel('Outlier Score')
    plt.ylabel('Frecuencia')
    plt.show()
    
    
def normalizar_scores(scores):
    scores = np.array(scores)
    
    scores = [math.exp(x) for x in scores]
    scores = np.round(scores,decimals=10)
    return scores


    
. 
def test_score (train,data_con,data_descon,solution,title = 'confussion matrix',mode = 'GA',percentile =100 ):
    Kern = KernelDensity(kernel=kernel_dict[int(solution[0])], bandwidth=solution[1]).fit(train)
    originals = Kern.score_samples(train)
    con_scores = Kern.score_samples(data_con)
    descon_scores = Kern.score_samples(data_descon)
    
   
    
    con_scores[con_scores == -np.inf] = np.min(con_scores[con_scores != -np.inf])
    descon_scores[descon_scores == -np.inf] = np.min(descon_scores[descon_scores != -np.inf])
    
    if mode == 'GA':

        percentile = 100-int(solution[2])
        threshold = np.percentile(originals, percentile) + solution[3] * np.std(originals)
    
    if mode == 'percentile':
        percentile = 100-percentile
        threshold = np.percentile(originals, percentile)
        
    outliers = np.where(originals > threshold)[0]
    inliers = np.where(originals <= threshold)[0]  


    T_P = len(np.where(con_scores > threshold)[0]) 
    F_N = len(np.where(con_scores < threshold)[0]) 
    F_P = len(np.where(descon_scores > threshold)[0]) 
    T_F = len(np.where(descon_scores < threshold)[0]) 
    #print ("TP: {} , TN: {} , FP: {} , FN: {}".format(T_P,T_F,F_P,F_N))
    
    #plot_confussion(T_P,T_F,F_P,F_N,title)
    return T_P,T_F,F_P,F_N



In [23]:
kernels = [1,2]
bandas = [0.1,0.3,0.4,0.5,0.6,0.8]
count = 0
results = pd.DataFrame(columns = ['Kernel','Banda','Precision','Recall','F1','Accuracy'])
for kernel in kernels:
    for banda in bandas: 
        solution = [kernel,banda]
        Precision,recall,F1,Acc = calculate_metrics(*test_score(Data_train,Data_val_con,Data_val_descon,solution,mode = 'percentile',percentile=100))
        results.loc[count] = [kernel_dict[kernel],banda,Precision,recall,F1,Acc]
        count += 1

In [24]:
results

,Kernel,Banda,Precision,Recall,F1,Accuracy
0,gaussian,0.1,0.000000,0.000000,0.000000,0.500000
1,gaussian,0.3,1.000000,0.021014,0.041164,0.510507
2,gaussian,0.4,1.000000,0.575362,0.730451,0.787681
3,gaussian,0.5,0.938416,0.927536,0.932945,0.933333
4,gaussian,0.6,0.741419,0.939130,0.828645,0.805797
5,gaussian,0.8,0.525646,0.987681,0.686131,0.548188
6,exponential,0.1,0.000000,0.000000,0.000000,0.500000
7,exponential,0.3,0.938825,0.834058,0.883346,0.889855
8,exponential,0.4,0.643559,0.948551,0.766842,0.711594
9,exponential,0.5,0.560453,0.967391,0.709729,0.604348


In [25]:
best = results.loc[results['Accuracy'].idxmax()]
best

Kernel       gaussian
Banda             0.5
Precision    0.938416
Recall       0.927536
F1           0.932945
Accuracy     0.933333
Name: 3, dtype: object

In [26]:
results.to_csv('Resultados/KDE_'+set+'_validation.csv')

In [27]:
solution = [best['Kernel'],best['Banda']]
if solution [0] == 'gaussian':
    solution [0] = 1
if solution [0] == 'exponential':
    solution [0] = 2
if solution [0] == 'epanechnikov':
    solution [0] = 3
    
Precision,recall,F1,Acc = calculate_metrics(*test_score(Data_train,Data_test_con,Data_test_descon,solution,mode = 'percentile',percentile=100))

resultado = pd.DataFrame(columns = ['Precision','Recall','F1','Accuracy'])

resultado.loc[0] = [Precision,recall,F1,Acc]
resultado.to_csv('Resultados/KDE_'+set+'_test.csv')
resultado


,Precision,Recall,F1,Accuracy
0,0.941301,0.964493,0.952756,0.952174


### Now the Genetic algorithm implementation

- [Documentation](https://pygad.readthedocs.io/en/latest/)

In [8]:
def GA_test(fitness_func,file_name,num_generations = 150,num_parents_mating = 4,sol_per_pop = 20,parent_selection_type = 'sss',keep_parents = 4,
            crossover_type = "two_points",mutation_type = "random",mutation_num_genes = 5,initial_population = None,parallel_processings = None):
    
    def callback_generation(ga_instance): # Fitness Function
        print("Generation = {generation}".format(generation=ga_instance.generations_completed))
        print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
        print ("Best individual: ", ga_instance.best_solution()[0])
        last_fitness = ga_instance.best_solution()[1]
        
    fitness_function = fitness_func

    num_genes = 4 # 

    random_mutation_min_val = -5
    random_mutation_max_val = 5
    if initial_population != None:
        initial_population =[initial_population] * sol_per_pop


        
    gene_space = [{'low':1,'high':3,'step':1},{'low': 0.05, 'high': 1,'step':0.01},{'low': 90, 'high': 100,'step':1},{'low': -5, 'high': 0,'step':0.01}]
    last_fitness = 0


    ga_instance = pygad.GA(num_generations=num_generations,
                            num_parents_mating=num_parents_mating,
                                fitness_func=fitness_function,
                                sol_per_pop=sol_per_pop,
                                num_genes=num_genes,
                               # init_range_low=init_range_low,
                               # init_range_high=init_range_high,
                                parent_selection_type=parent_selection_type,
                                crossover_type=crossover_type,
                                mutation_type=mutation_type,
                                mutation_num_genes=mutation_num_genes,
                                gene_space=gene_space,
                                allow_duplicate_genes=False,
                                keep_elitism= keep_parents,
                                parallel_processing= parallel_processings,
                                initial_population=initial_population,
                                on_generation=callback_generation)
        
    ga_instance.run()
    
    save_ga(ga_instance,file_name)
    
    
path_save_ga = 'GA_results/'

In [9]:
def fitness_func(ga,solution,solution_idx):
    results = test_score(Data_train,Data_val_con,Data_val_descon,solution)
    Precision,Recall,F1,Acurracy = calculate_metrics(*results)
    
    return Acurracy
 
GA_test(fitness_func,path_save_ga+'Ga_KDE_'+set,num_generations = 20,sol_per_pop=6,mutation_num_genes=2,
        keep_parents=1,initial_population=[1,0.5,100,0])


KeyboardInterrupt: 

In [9]:
solution = pygad.load(path_save_ga+'Ga_KDE_'+set).best_solution()[0]

Precision,recall,F1,Acc = calculate_metrics(*test_score(Data_train,Data_test_con,Data_test_descon,solution))
print ("Precision: {} , Recall: {} , F1: {} , Accuracy: {}".format(Precision,recall,F1,Acc))

resultado = pd.DataFrame(columns = ['Precision','Recall','F1','Accuracy'])

resultado.loc[0] = [Precision,recall,F1,Acc]

resultado.to_csv('Resultados/KDE_'+set+'_test_GA.csv')

Precision: 0.9919649379108838 , Recall: 0.9840579710144928 , F1: 0.9879956347762823 , Accuracy: 0.9880434782608696
